# Descrição do problema

Usuários da eNB1 se afastam da mesma em linha reta e com um ângulo aleatório, enquanto realizam o download de um vídeo de 15 MB. Eventualmente, devido ao enfraquecimento do sinal da sua eNB, eles fazem handover para a eNB2 ou eNB3 através do algoritmo A3RSRP. Com acesso aos **sinais de referência** das 3 eNBs no momento do handover, use redes neurais para prever em qual **região** se encontra cada usuário e qual **vazão** possuem os mesmos nos primeiros 100 segundos de simulação.
<img src="ang.png" style="width: 400px">

 # Parte 2 - Classificação
<font color='blue'>Problemas de classificação são aqueles cujo desafio é prever o valor de uma variável dependente categórica, tal como a variável **region** deste dataset. Nesta etapa do HandsOn, você deverá criar uma rede neural usando as bibliotecas sklearn, keras e tensorflow para classificar corretamente esta variável.</font>

Obs.: Antes de criar os classificadores, execute o código criado na parte 1.

## Usando sklearn
Além de fornecer todas as funções que utilizadas neste hands on para processamento dos dados, seleção de modelos e testes de desempenho, a biblioteca sklearn também permite a criação de redes neurais de forma simplificada, sendo necessário apenas instanciar objetos das classes **MLPClassifier** (classificação) e **MLPRegressor** (regressão) para fazê-lo. 

### Parâmetros importantes do <a href="http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html">**MLPClassifier**</a> :
* **hidden_layer_sizes** (número de neurônios nas camadas escondidas)
* **solver** (algoritmo de otimização usado para o aprendizado)
    * lbfgs
        * *Limited-memory Broyden–Fletcher–Goldfarb–Shanno*.
        * Método da família quase-Netwon.
    * adam 
        * *Adaptive moment estimation*.
        * Método estocástico de primeira ordem.
        * <a href="https://arxiv.org/pdf/1412.6980.pdf"> Descrição do método</a>.
    * sgd 
        * *Stochastic gradient descent*.
        * Método estocástico de primeira ordem.
* **activation** (função de ativação para as camadas escondidas)
    * relu 
        * $ f(x) = max(0,x) $
    * tanh 
        * $ f(x) = tanh(x) $
    * identity 
        * $ f(x) = x $
    * logistic 
        * $ f(x) =  1/(1 + e^{-x})$
* **learning_rate_init** (passo de aprendizado inicial)
    * Apenas usado em otimizadores estocásticos.
    * Este passo pode ser alterado durante o treinamento de acordo com o otimizador usado. Consulte a documentação para mais esclarecimentos.
* **batch_size** (tamanho dos minibatches)
    * Número de amostras que passam pela rede para cada ajuste de pesos.
    * Só é usada para otimizadores estocásticos.
* **max_iter** (número máximo de épocas de treinamento)
* **early_stopping** (parada antecipada)
    * Ao definir esta variável como *True*, o processo de ajuste de pesos estará sujeito a terminar de forma antecipada, caso o score no **conjunto de validação** não melhore por pelo menos **tol** (valor definido para a tolerância) em duas **épocas de treinamento** consecutivas. 
    * Esta variável melhora a rapidez do processamento da rede para otmizadores estocásticos.
    * Definir essa variável como *True* não é eficaz se o solver for definido como lbfgs.
* **validation_fraction**
    * Define um percentual do conjunto de treino que será usado ao final de cada **época de treinamento** para medir a performance da rede, caso **early_stopping** seja definido como *True*.
    * Este subconjunto, chamado de **conjunto de validação**, não é usado diretamente no ajuste de pesos, representando um conjunto de testes provisório.
* **random_state** (estado aleatório)
    * Semente de aleatoriedade que controla todos os processos aletaórios usados na criação da rede.
    * Definir esta variável não é essencial, mas garante a reprodutibilidade dos resultados obtidos para um determinado conjunto de parâmetros.

In [ ]:
# Importando a classe que permite criar percéptrons de múltiplas camadas para tarefas de classificação
from sklearn.neural_network import MLPClassifier

# Criando uma rede neural com 3 camadas escondidas
classifier_sk=MLPClassifier(random_state=0, solver='lbfgs', hidden_layer_sizes=[100,50,25])

Com os objetos já instanciados, a função *fit*, presente em ambas as classes, realiza todo o processo de apendizado, estando as redes, após a aplicação deste função, prontas para serem testadas.

In [ ]:
# Ajustando os pesos da rede ao conjunto de treino
classifier_sk.fit(x_train,y_train)

# Aplicando a rede já treinada ao conjunto de testes
# Observe aue a veriável independente não é passada como parâmetro
y_pred=classifier_sk.predict(x_test)

### Avaliação dos resultados
Para avaliar os resultados de um problema de classificação, usaremos uma ferramenta chamada **matriz de confusão**. Esta matriz é útil por mostrar não somente a precisão de cada modelo, mas também os tipos de erro cometido, sendo possível identificar, por exemplo, se a rede tende a classificar os exemplos mais em uma classe do que nas outras.
Observe a figura abaixo: <img src="confusionMatrix.png" style="width: 400px"> 

In [ ]:
# Importando a função que permite avaliar a matriz de confusao
from sklearn.metrics import confusion_matrix

# Revertendo o OneHotEncoder (para a criação da matriz de confusao) com a
## multiplicacao da matriz de previsões pelo vetor [0,1,2]
y_pred_rev=np.matmul(y_pred,[0,1,2])
y_test_rev=np.matmul(y_test,[0,1,2])

# Criando a matriz de confusao
cm = confusion_matrix(y_test_rev, y_pred_rev)

# Resultado
print('Matriz de Confusão\n'+str(cm))

### Exercício 1
Faça uma rede com o **solver sgd** e com parada antecipada. Defina o **learning_rate_init** como 1 e veja a matriz de confusão da máquina. Por que isso ocorreu?

### Exercício 2
Teste outras topologias para a rede. Qual é o número mínimo de neurônios a conseguir um resultado 100%? 

## Usando keras + tensorflow

### Inicializando a rede neural
* A criação da rede neural pela biblioteca keras, que usa o tensorflow como backend, tem a vantagem de permitir um ajuste mais fino dos parâmetros do que a sklearn, mantendo, todavia, um nível baixo de complexidade.
* Esta biblioteca permite que se ajuste os detalhes de cada camada individualmente, sendo a rede definida como uma **sequência de camadas**. Por este motivo, a rede abaixo deverá ser inicializada como um objeto da classe *Sequential*. 

In [10]:
# Importando a biblioteca Keras
import keras

# Importando a classe usada para inicializar a rede neural
from keras.models import Sequential

# Inicializando a rede neural
classifier_kt = Sequential()

Using TensorFlow backend.


### Criando as camadas da rede
* Cada camada tem como parâmetros o número de entradas (**input_dim**), o número de saídas (**output_dim**), os pesos iniciais (**init**) e a função de ativação da camada (**activation**).
* O parâmetro **input_dim** só necessita ser explicitado na primeira camada, na qual representa o número de inputs. Nas demais, fica subentendido que o número de entradas é igual ao número de saídas da camada anterior.
* O **output_dim** é igual ao número de neurônios de cada camada. Na cadama de saída, este parâmetro deve ser igual à quantidade de outputs da rede ou ao número de dummy variables, caso se esteja lidando com uma variável que usou o *OneHotEncoder*.
* Para a camada de saída num problema de classificação, a função de ativação recomendada é *softmax*, caso se esteja lidando com dummy variables, ou *sigmoid* caso contrário.
    * Ambas as funções não retornam valores binários, mas probabilidades, portanto precisam ser convertidas em números binários ao final do processo.
    * A diferença fundamental entre elas reside no fato de que a *sigmoid* analisa as variáveis individualmente, permitindo que mais de um valor seja igual a 1 ou que todos sejam iguais a 0.
* É desejável que os pesos sejam inicializados como números aleatórios pequenos. Portanto, aqui se usará um gerador de números aleatórios com uma distribuição uniforme entre -0,5 e 0,5.

In [ ]:
# Importando a classe usada para criar as camadas da rede
from keras.layers import Dense 

# Criando a variável que inicializará os pesos
init=keras.initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=0)

# Criando a camada de entrada e a primeira camada escondida da rede
classifier_kt.add(Dense(output_dim=100,init=init,activation='relu', input_dim=6))

# Criando a segunda camada escondida da rede
classifier_kt.add(Dense(output_dim=50,init=init,activation='relu'))

# Criando a terceira camada escondida da rede
classifier_kt.add(Dense(output_dim=25,init=init,activation='relu'))

# Criando a camada de saida
classifier_kt.add(Dense(output_dim=3,init=init,activation='softmax'))

### Ajustando a rede neural ao conjunto de treino
* Com o formato da rede já definido, agora é necessário definir os parâmetros que serão usados no ajuste dos pesos:
    * **optimizer**
        * Método de otimização usado.
        * Tem função idêntica ao parâmetro *solver* no MLPClassifier.
        * <a href="https://keras.io/optimizers/">Lista dos métodos disponíveis</a>.
    * **loss**
        * Função objetivo da otimização.
        * A mais comum para problemas de classificação binária é a *binary_crossentropy*.
        * Já para problemas com mais de 2 categorias, a mais recomendada é a função *categorical_crossentropy*
        * O MLPClassifier não possui parâmetro ajustável equivalente.
* Além desses dois parâmetros obrigatórios, também há o parâmetro **metrics**, que é uma função usada para avaliar a performance do modelo. Ela se diferencia da **loss** por não ser usada no ajuste dos pesos, sendo apenas uma métrica adicional para a vizualização do programador durante o processo de treinamento.

In [ ]:
# Compiling a rede neural
classifier_kt.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Ajustando a rede neural ao conjunto de treino
classifier_kt.fit(x_train,y_train, epochs=100, batch_size=100)

In [ ]:
# Realizando a previsão de y
y_pred=classifier_kt.predict(x_test)
# Transformando os valores previstos em binário
y_pred = (y_pred > 0.5)

### Avaliação dos resultados

In [ ]:
# Importando a função que permite avaliar a matriz de confusão
from sklearn.metrics import confusion_matrix

# Revertendo o OneHotEncoder (para a criação da matriz de confusão) com a
## multiplicação da matriz de previsões pelo vetor [0,1,2]
y_pred_rev=np.matmul(y_pred,[0,1,2])
y_test_rev=np.matmul(y_test,[0,1,2])

# Criando a matriz de confusão
cm = confusion_matrix(y_test_rev, y_pred_rev)

# Resultado
print('Matriz de Confusão\n'+str(cm))

### Exercício 3
Usando a rede criada com o keras, varie os parâmetros **epochs** e **batch_size** e veja o impacto dos mesmos no desempenho da rede.

# Parte 3 - Regressão
<font color='blue'>Problemas de regressão são aqueles cujo desafio é prever o valor de uma variável dependente não categórica, tal como a variável **throughput** deste dataset, que pode assumir qualquer valor no intervalo de de 0 a 4,1 Mbps. Nesta etapa do HandsOn, você deverá criar uma rede neural usando as bibliotecas sklearn, keras e tensorflow para tentar achar o valor desta variável.</font>

Obs.: Antes de criar os regressores, execute o código criado no exercício 1 da parte 1.

### Output scaling
* No problema anterior, a padronização da escala do output não foi mencionada, pois o mesmo já se encontrava em formato binário. Apesar de não obrigatória, essa padronização tende a melhorar a performance da rede quando o intervalo de valores possivelmente assumidos por y é muito extenso.
* Todavia, quando saber o valor do output na escala original é necessário, **a padronização deve ser revertida após a saída da rede neural**.
* Observe que apenas o y_train é passado à máquina, portanto y_test não precisa ser padronizado.

In [9]:
# Criando um novo scaler
fs_y=StandardScaler()

# Modificando o formato do vetor de outputs
y_train=np.array(y_train)
y_train=y_train.reshape(-1, 1)

y_test=np.array(y_test)
y_test=y_test.reshape(-1, 1)

# Transformando e ajustando apenas o conjunto de treinos
y_train=fs_y.fit_transform(y_train)

# Resultado nas 4 primeras linhas
print(y_test[0:4,:])

[[3.74434595]
 [3.25324784]
 [3.96798345]
 [3.79998007]]


## Usando sklearn

### Exercício 1
Sabendo que os parâmetros do <a href="http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html">**MLPRegressor**</a> são idênticos aos do MLPClassifier, crie e treine uma rede neural para encontrar o **throughput** com esta classe. Use os parâmetros que desejar.


### Avaliação dos resultados
Para avaliar os resultados dos problemas de regressão, usaremos a função de erro médio quadrático, cuja fórmula encontra-se abaixo:

####  <center>  $ E_{mse} = \sum_{i=1}^{N}\frac{(y_{real}-y_{pred})}{N}$  </center> 

In [ ]:
# Importando a função que permite calcular o erro médio quadrático
from sklearn.metrics import mean_squared_error

# Invertendo o Standard Scaler
y_pred=fs_y.inverse_transform(y_pred)

# Calculando o erro médio quadrático
mse=mean_squared_error(y_test, y_pred)

# Resultado
print('Erro médio quadrático: '+str(mse))

### Exercício 2
Usando a rede do exercício 1, desfaça a padronização do output e verifique o novo erro médio quadrático. Qual foi o impacto desta padronização no desempenho da rede?

## Usando keras + tensorflow

* Assim como no sklearn, os parâmetros do keras + tensorflow também são os mesmos que os da tarefa de classificação. Há apenas algumas ressalvas a serem feitas:
    * A **activation** da última camada deve ser adequada à natureza do problem. A mais comum para problemas de regressão é a *linear*, que permite que o output assuma qualquer valor, mas, por exemplo, se o seu output só possui valores positivos, a função *relu* lhe trará melhores resultados;
    * A função **loss** mais usada para problemas de regressão é a *mean_squared_error* (erro médio quadrático).    


### Exercício 3
Crie e treine uma rede neural para encontrar o **throughput** dos usuários usando o keras. Avalie o desempenho da rede.

Dica: Se lembre de ajustar o número de outputs na camada de saída. 